### CS224N 2020 Winter P4

### 1. NMT Seq2Seq

#### 1.g Use of masks
* what effect the masks have on the entire attention computation:   

  In attention computation, $e_{t,i}=(h_t^{dec})^T W_{attProj}h_i^{enc}$ will produce a T dimentional vector, with the ith component corresponds to the ith word in a sentence. The mask vector 'enc_masks' will put a '1' for each padded position, and it's embedding e_t is set to -inf where enc_masks has 1. It has the effect of having an attention 0 for each padded position.  
  
  

* why it is necessary to use the masks in this way:  
  
  Apart from achieving 0 attention for the padded position, the necessarty of using masks in this way is primarily because we use batch gradient descent during training, and by making each sentence the same length with padded words and masks the training can be done by GPU parallel computation. (Another thoughts of improving parallelization is to sample sentences with similar lengths into batches, hence reducing the number of padded position and improves the efficiency of training.)

#### 1.i BLEU output


#### 1.j Attention comparison
* advantage and disadvantage of dot product attention compared to multiplicative attention
  Dot product attention $e_t=(h_t^{dec})^T h^{enc}$ has the a much smaller amount of parameters because it has no attention projection matrix $W_{attProj}$.  
  The limitation of Dot product attention is that embeddings in both encoder and decoder need to have the same dimension, which is not required in multiplication attention.   
  
  
* advantage and disadvantage of additive attention compared to multiplicative attention   
  Additive attention has better performance for larger dimensions; the hyper parameter can be adjusted in order to avoid high dimension computation, for example the dimension of $v^T$ can be chosen a smaller number so as to achieve a better performance. 
  
  In the case of smaller hidden state dimensionality, additive attention has a much more complex training model because it brings non-linearity into attention computation. 

### 2. Analyzing NMT

### 2.a
1. Here’s another favorite of my favorites, “The Starry Night”.
  * Problem:
  * Reason:
  * Solution:
  
  
2. You know what I do is write for children, and in fact, I’m probably the author for children, more reading in the U.S.  
  * Problem:
  * Reason:
  * Solution:
  
  
3. A friend of mine did that – Richard $\text{<unk>}$.     
  * Problem: failed to translate OOV words.
  * Reason: OOV words are missing from the embedding matrix.
  * Solution: In this particular case, the OOV problem with people's names can be fixed or mitigated by an NER task then simply copy the OOV name to the destination. Another possible solution is to learn OOV word embeddings from dictionary definitions, such as dict2vec.    
  
    
4. You just have to go back to the apple to see it as an epiphany.  
  * Problem: Failed to generate phrase/idioms
  * Reason: phrase/idioms are not in the embedding matrix and hence cannot be predicated as a single unit. 
  * Solution: 'go around the block' is a phrase meaning 'to have a lot of experience'. 'go back to the apple' in english does not make much sense. One way that can mitigate the problem is to also embed phrase/idioms and learning it as a single embedding.      
    
    
5. She saved my life by letting me go to the bathroom in the women's room.  
  * Problem: Failed to predict gender info in translation.
  * Reason: In Spanish a lot of the words have different masculine or feminine forms; In this case, the NMT translation first failed in predicting the gender of words, and further translates 'profesores' to 'women' which is likely the consequence incorrect gender inference. 
  * Solution: 
  
  
  
6. That’s over 100,000 acres.  
  * Problem: Failed to deal with different measurement system.
  * Reason: 
  * Solution: 

#### 2.b 
1. Error 1
    * src: 
    * ref:
    * nmt:
    * error:
    * reason:
    * solution:
    

2. Error 2
    * src:
    * ref:
    * nmt:
    * error:
    * reason:
    * solution:


#### 2.c BLEU score
1. 
2. 
3. 
4. 